In [1]:
import os
import scanpy as sc

In [2]:
import glob
import numpy as np

In [3]:
os.listdir()

['Untitled.ipynb',
 'zheng68k',
 'run_data_prep.ipynb',
 '.ipynb_checkpoints',
 'COVID',
 'run_seurat_ref_mapping.ipynb',
 'Mye',
 'ms',
 'NSCLC']

In [4]:
# for COVID:
prefix = "NSCLC"
ORI_INPUT_DIR = "./"+prefix+"/"

Train_INPUT_FILE = glob.glob(ORI_INPUT_DIR+"*_train*.h5ad")[0]
Test_INPUT_FILE = glob.glob(ORI_INPUT_DIR+"*_test*.h5ad")[0]

file_paths = [Train_INPUT_FILE, Test_INPUT_FILE]

file_names = list(map(lambda path: os.path.basename(path), file_paths))
#file_names_without_extension = list(map(lambda name: os.path.splitext(name)[0], file_names))

# Assigning file names to specific variables
train_name, test_name = file_names

prefix = os.path.splitext(train_name)[0].split('_')[0]


In [19]:
train_name
print(Train_INPUT_FILE)

./NSCLC/NSCLC_train_Raw.h5ad


In [6]:
import scanpy as sc
train_ad = sc.read_h5ad(Train_INPUT_FILE)
test_ad = sc.read_h5ad(Test_INPUT_FILE)

In [7]:
train_ad

AnnData object with n_obs × n_vars = 30812 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'n_counts', 'cell_type'
    var: 'ensembl_id', 'features', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'

In [93]:
train_ad.X[:50,-15:].todense()

matrix([[0.        , 2.80359526, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 2.33489334, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 4.37102888, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.       

In [28]:
from scipy import sparse, io


In [91]:
sc.pp.normalize_total(train_ad, target_sum=1e4)
sc.pp.log1p(train_ad)

In [92]:
train_ad

AnnData object with n_obs × n_vars = 30812 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'n_counts', 'cell_type'
    var: 'ensembl_id', 'features', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg', 'log1p'

In [94]:
io.mmwrite(ORI_INPUT_DIR+prefix+"_train.mtx", train_ad.X)

In [8]:
#train_ad.obs["cell_type"] = train_ad.obs["celltype"]
meta_df = train_ad.obs[[ 'cell_type']]

In [21]:
meta_df.head()

,cell_type
P1.tr.1.AAACCTGCACATGACT-1,CD4_C8-Treg
P1.tr.1.AAACCTGCAGATCGGA-1,CD4_C8-Treg
P1.tr.1.AAACCTGGTCGGATCC-1,CD4_C8-Treg
P1.tr.1.AAACCTGTCTGTCAAG-1,NA
P1.tr.1.AAACGGGGTAGTACCT-1,CD4_C8-Treg


In [10]:
set(list(meta_df['cell_type']))

{'CD4_C1-Naive',
 'CD4_C2-Tcm',
 'CD4_C3-Tem',
 'CD4_C4-CD69',
 'CD4_C5-ISG15',
 'CD4_C6-RPL',
 'CD4_C7-Th1-like',
 'CD4_C8-Treg',
 'CD4_C9-Prolif.',
 'NA',
 'Non-exhausted',
 'Prolif.',
 'Tex',
 'XCL1'}

In [15]:
set(list(test_ad.obs[['cell_type']]['cell_type'])) == set(list(meta_df['cell_type']))

True

In [97]:
meta_df.to_csv(ORI_INPUT_DIR+prefix+"_train_meta.csv", index=True)

In [98]:
train_ad.var['gene_name'] = train_ad.var_names.tolist()
train_ad.var[['gene_name']].to_csv(ORI_INPUT_DIR+prefix+"_train_genes.csv", index=True)

In [99]:
train_ad.var[['gene_name']].head()

,gene_name
features,
HES4,HES4
ISG15,ISG15
TNFRSF18,TNFRSF18
TNFRSF4,TNFRSF4
PRDM16,PRDM16


In [17]:
test_ad

AnnData object with n_obs × n_vars = 38515 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'n_counts', 'cell_type'
    var: 'ensembl_id', 'features', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'

In [101]:
sc.pp.normalize_total(test_ad, target_sum=1e4)
sc.pp.log1p(test_ad)

In [104]:
io.mmwrite(ORI_INPUT_DIR+prefix+"_test.mtx", test_ad.X)#.layers['logcounts'])
#test_ad.obs["cell_type"] = test_ad.obs["celltype"]
meta_df = test_ad.obs[['cell_type']]
meta_df = test_ad.obs[['cell_type']]
meta_df.to_csv(ORI_INPUT_DIR+prefix+"_test_meta.csv", index=True)
test_ad.var['gene_name'] = test_ad.var_names.tolist()
test_ad.var[['gene_name']].to_csv(ORI_INPUT_DIR+prefix+"_test_genes.csv", index=True)

In [16]:
30812+38515

69327